In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:
import pandas as pd
from tqdm import tqdm

In [2]:
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.screening_modes_api import ScreeningModesApi

from exact_sync.v1.models import ImageSet, Team, Image, ScreeningMode
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [4]:
configuration = Configuration()
configuration.username = 'marzahl'
configuration.password = 'imagetagger'
configuration.host =  "https://exact.cs.fau.de/"

In [6]:
client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
images_api = ImagesApi(client)
screening_api = ScreeningModesApi(client)

In [22]:
data = []
µm_pixel = 0.25

for image_set_id in tqdm([246, 245, 251, 15, 222, 247]):
    for image in images_api.list_images(pagination=False, image_set=image_set_id).results:
        
        area = (image.width * image.height) / 1000000 * (µm_pixel * µm_pixel)
        for screening in screening_api.list_screening_modes(image=image.id, user=12, omit="screening_tiles").results:         
            data.append([image.name, image.id, image_set_id, image.height, 
                         image.width, screening.x_steps, screening.y_steps, screening.x_steps * screening.y_steps, area])
            
            break
        
imageStat = pd.DataFrame(data, columns=["name", "id", "image_set", "height", "width", "x_steps", "y_steps", "patches", "area"])
imageStat    

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:20<00:00,  3.34s/it]


,name,id,image_set,height,width,x_steps,y_steps,patches,area
0,07_EIPH_574882 Berliner Blau-001.svs,3530,246,32990,37799,22,33,726,77.936813
1,08_EIPH_574999 Berliner Blau.svs,3536,246,36678,37799,22,36,792,86.649483
2,09_EIPH_563417 L berliner blau.svs,3539,246,40195,41399,24,40,960,104.002050
3,10_EIPH_563417 R Berliner Blau.svs,3543,246,37633,39599,23,37,851,93.139323
4,13_EIPH_570370 Berliner Blau.svs,3545,246,32358,34200,20,32,640,69.165225
...,...,...,...,...,...,...,...,...,...
62,25_EIPH_568150 Turnbull blue.svs,3569,247,32365,32399,13,22,286,65.537102
63,26_EIPH_566482 L Turnbull blue.svs,3570,247,33163,32399,14,24,336,67.153002
64,27_EIPH_571557 Turnbull blue.svs,3572,247,31881,32399,19,32,608,64.557032
65,30_EIPH_568355 Turnbull blue.svs,3574,247,31878,34200,14,21,294,68.139225


In [23]:
imageStat["patches"].sum()

42909

In [24]:
imageStat["area"].sum()

4726.012673125